<a href="https://colab.research.google.com/github/JaySanthanam/Programming-for-data/blob/main/Projects/Work_in_progress/Automate_differences_version2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Test code for automated output file (version 2). Is there a simplier and more reliable way to find differences between 2 dataframes of differing sizes?

#Load files 

##new_df is new data including new year's data
##old_df has old data

In [72]:
import pandas as pd
import numpy as np

def get_data(url): # this function gets data stored as csv file from a given url
  df = pd.read_csv(url)
  return df

url_new = "https://raw.githubusercontent.com/JaySanthanam/Programming-for-data/main/Datasets/LGBF_Data_Table_May.csv"
new_df = get_data(url_new)
url_old = "https://raw.githubusercontent.com/JaySanthanam/Programming-for-data/main/Datasets/LGBF_Data_Table_March.csv"
old_df = get_data(url_old)


#Check if files are loaded correctly

check no. of rows and columns

In [73]:
old_df.shape

(31160, 6)

check column names

In [74]:
old_df.head()

,Code,LocalAuthority,Year,Indicator,Numerator,Denominator
0,C&L01,Aberdeen City,2010-11,0.406679,785.018799,1922292.0
1,C&L01,Aberdeen City,2011-12,0.922524,1886.603428,2045051.0
2,C&L01,Aberdeen City,2012-13,4.335545,9381.043169,2163756.0
3,C&L01,Aberdeen City,2013-14,4.044078,8988.323761,2222588.0
4,C&L01,Aberdeen City,2014-15,3.476122,8645.573117,2487138.0


check no. of rows and columns and column names in new_df

In [75]:
new_df.shape

(31358, 6)

In [76]:
new_df.head()

,Code,LocalAuthority,Year,Indicator,Numerator,Denominator
0,C&L01,Aberdeen City,2010-11,0.406679,785.018799,1922292.0
1,C&L01,Aberdeen City,2011-12,0.922524,1886.603428,2045051.0
2,C&L01,Aberdeen City,2012-13,4.335545,9381.043169,2163756.0
3,C&L01,Aberdeen City,2013-14,4.044078,8988.323761,2222588.0
4,C&L01,Aberdeen City,2014-15,3.476122,8645.573117,2487138.0


#The column values are not exactly in the same format between the two dataframes. So let's make it consistent.

In [77]:
old_df.Indicator = old_df.Indicator.round(4)
old_df.Numerator = old_df.Numerator.round(4)
old_df.Denominator = old_df.Denominator.round(4)

#same for new_df

new_df.Indicator = new_df.Indicator.round(4)
new_df.Numerator = new_df.Numerator.round(4)
new_df.Denominator = new_df.Denominator.round(4)


#Merge Two Dataframes of Different Lengths with Matching ID and Fill Missing Values

We will use Indicator code, local authority name and year as ids to match the data between two dataframes. If one of them has new year data, we will create a new line with nan in the old dataframe.

In [78]:
df = old_df.merge(new_df, on=['Code','LocalAuthority','Year'], how='outer', indicator=True, suffixes=['_old', '_new']).set_index(['Code','LocalAuthority','Year']) 
df

Indicator_old  Numerator_old  Denominator_old  \
Code   LocalAuthority Year                                                     
C&L01  Aberdeen City  2010-11         0.4067       785.0188        1922292.0   
                      2011-12         0.9225      1886.6034        2045051.0   
                      2012-13         4.3355      9381.0432        2163756.0   
                      2013-14         4.0441      8988.3238        2222588.0   
                      2014-15         3.4761      8645.5731        2487138.0   
...                                      ...            ...              ...   
CHN08b Scotland       2021-22            NaN            NaN              NaN   
CHN09  Scotland       2021-22            NaN            NaN              NaN   
CHN17  Scotland       2021-22            NaN            NaN              NaN   
CHN22  Scotland       2021-22            NaN            NaN              NaN   
CHN23  Scotland       2021-22            NaN            NaN              NaN   

                               Indicator_new  Numerator_new  Denominator_new  \
Code   LocalAuthority Year                                                     
C&L01  Aberdeen City  2010-11         0.4067       785.0188        1922292.0   
                      2011-12         0.9225      1886.6034        2045051.0   
                      2012-13         4.3355      9381.0432        2163756.0   
                      2013-14         4.0441      8988.3238        2222588.0   
                      2014-15         3.4761      8645.5731        2487138.0   
...                                      ...            ...              ...   
CHN08b Scotland       2021-22       402.2039    236586.8746          11312.0   
CHN09  Scotland       2021-22         0.8981            NaN              NaN   
CHN17  Scotland       2021-22         0.8214            NaN              NaN   
CHN22  Scotland       2021-22         0.0801            NaN              NaN   
CHN23  Scotland       2021-22         0.1586            NaN              NaN   

                                   _merge  
Code   LocalAuthority Year                 
C&L01  Aberdeen City  2010-11        both  
                      2011-12        both  
                      2012-13        both  
                      2013-14        both  
                      2014-15        both  
...                                   ...  
CHN08b Scotland       2021-22  right_only  
CHN09  Scotland       2021-22  right_only  
CHN17  Scotland       2021-22  right_only  
CHN22  Scotland       2021-22  right_only  
CHN23  Scotland       2021-22  right_only  

[31358 rows x 7 columns]

##New year's data (extra rows in new_df) will be the ones that are not found in both dataframes. Since we want to keep this more generalised and sometimes the old dataframe can have extra rows. New year's data are the extra rows in new_df which was joined on the right (see above code). So select the extra rows in new_df, we will look for 'right_only' in the '_merge' column 

In [79]:
new_year = df.loc[df['_merge'] == 'right_only']
new_year

Indicator_old  Numerator_old  \
Code   LocalAuthority   Year                                    
CHN08a Aberdeen City    2021-22            NaN            NaN   
       Aberdeenshire    2021-22            NaN            NaN   
       Angus            2021-22            NaN            NaN   
       Argyll & Bute    2021-22            NaN            NaN   
       Clackmannanshire 2021-22            NaN            NaN   
...                                        ...            ...   
CHN08b Scotland         2021-22            NaN            NaN   
CHN09  Scotland         2021-22            NaN            NaN   
CHN17  Scotland         2021-22            NaN            NaN   
CHN22  Scotland         2021-22            NaN            NaN   
CHN23  Scotland         2021-22            NaN            NaN   

                                 Denominator_old  Indicator_new  \
Code   LocalAuthority   Year                                      
CHN08a Aberdeen City    2021-22              NaN      4154.9845   
       Aberdeenshire    2021-22              NaN      2579.0232   
       Angus            2021-22              NaN      3505.4415   
       Argyll & Bute    2021-22              NaN      2931.0285   
       Clackmannanshire 2021-22              NaN      4729.8493   
...                                          ...            ...   
CHN08b Scotland         2021-22              NaN       402.2039   
CHN09  Scotland         2021-22              NaN         0.8981   
CHN17  Scotland         2021-22              NaN         0.8214   
CHN22  Scotland         2021-22              NaN         0.0801   
CHN23  Scotland         2021-22              NaN         0.1586   

                                 Numerator_new  Denominator_new      _merge  
Code   LocalAuthority   Year                                                 
CHN08a Aberdeen City    2021-22            NaN              NaN  right_only  
       Aberdeenshire    2021-22            NaN              NaN  right_only  
       Angus            2021-22            NaN              NaN  right_only  
       Argyll & Bute    2021-22            NaN              NaN  right_only  
       Clackmannanshire 2021-22            NaN              NaN  right_only  
...                                        ...              ...         ...  
CHN08b Scotland         2021-22    236586.8746          11312.0  right_only  
CHN09  Scotland         2021-22            NaN              NaN  right_only  
CHN17  Scotland         2021-22            NaN              NaN  right_only  
CHN22  Scotland         2021-22            NaN              NaN  right_only  
CHN23  Scotland         2021-22            NaN              NaN  right_only  

[198 rows x 7 columns]

There are 198 new data in new_df

If there are extra rows in old_df that doesn't exist in new_df we can check using 'left_only' in the above code.

In [80]:
old_only = df.loc[df['_merge'] == 'left_only']
old_only

,,,Indicator_old,Numerator_old,Denominator_old,Indicator_new,Numerator_new,Denominator_new,_merge
Code,LocalAuthority,Year,,,,,,,


As expected there are none.

#Let's now compare the common data between old_df and new_df

In [81]:
both_df = df.loc[df['_merge'] == 'both']
both_df

Indicator_old  Numerator_old  Denominator_old  \
Code  LocalAuthority Year                                                     
C&L01 Aberdeen City  2010-11         0.4067       785.0188        1922292.0   
                     2011-12         0.9225      1886.6034        2045051.0   
                     2012-13         4.3355      9381.0432        2163756.0   
                     2013-14         4.0441      8988.3238        2222588.0   
                     2014-15         3.4761      8645.5731        2487138.0   
...                                     ...            ...              ...   
SW07  West Lothian   2017-18         0.8707            NaN              NaN   
                     2018-19         0.8542            NaN              NaN   
                     2019-20         0.8125            NaN              NaN   
                     2020-21         0.8430            NaN              NaN   
                     2021-22         0.8075            NaN              NaN   

                              Indicator_new  Numerator_new  Denominator_new  \
Code  LocalAuthority Year                                                     
C&L01 Aberdeen City  2010-11         0.4067       785.0188        1922292.0   
                     2011-12         0.9225      1886.6034        2045051.0   
                     2012-13         4.3355      9381.0432        2163756.0   
                     2013-14         4.0441      8988.3238        2222588.0   
                     2014-15         3.4761      8645.5731        2487138.0   
...                                     ...            ...              ...   
SW07  West Lothian   2017-18         0.8707            NaN              NaN   
                     2018-19         0.8542            NaN              NaN   
                     2019-20         0.8125            NaN              NaN   
                     2020-21         0.8430            NaN              NaN   
                     2021-22         0.8075            NaN              NaN   

                             _merge  
Code  LocalAuthority Year            
C&L01 Aberdeen City  2010-11   both  
                     2011-12   both  
                     2012-13   both  
                     2013-14   both  
                     2014-15   both  
...                             ...  
SW07  West Lothian   2017-18   both  
                     2018-19   both  
                     2019-20   both  
                     2020-21   both  
                     2021-22   both  

[31160 rows x 7 columns]

No. of rows matches with old_df.

##For testing!!
We only want the rows that have different data between old and new values in the both_df

In [82]:
old_diff = both_df.loc[(both_df['Indicator_new'] != both_df['Indicator_old'])]
old_diff

Indicator_old  Numerator_old  \
Code  LocalAuthority      Year                                    
C&L01 Angus               2021-22         6.2791      3551.1012   
      Clackmannanshire    2021-22         5.2879       610.5016   
      East Ayrshire       2010-11         2.5983      3094.4776   
      Scotland            2019-20         2.8858    150191.5886   
                          2021-22         6.4339    170577.7577   
...                                          ...            ...   
SW05  South Ayrshire      2021-22       427.0388     19707.9125   
      South Lanarkshire   2021-22       707.5046     63095.4953   
      Stirling            2021-22       607.6322     12480.8119   
      West Dunbartonshire 2021-22       920.2790     22970.2494   
      West Lothian        2021-22       708.7398     28285.9103   

                                   Denominator_old  Indicator_new  \
Code  LocalAuthority      Year                                      
C&L01 Angus               2021-22         565544.0         6.2985   
      Clackmannanshire    2021-22         115452.0         4.8465   
      East Ayrshire       2010-11        1190983.0         2.6003   
      Scotland            2019-20       52045545.0         2.8610   
                          2021-22       26512151.0         6.4324   
...                                            ...            ...   
SW05  South Ayrshire      2021-22            887.5       670.7826   
      South Lanarkshire   2021-22           1715.0       860.0207   
      Stirling            2021-22            395.0       752.9366   
      West Dunbartonshire 2021-22            480.0       871.3208   
      West Lothian        2021-22            767.5       751.2256   

                                   Numerator_new  Denominator_new _merge  
Code  LocalAuthority      Year                                            
C&L01 Angus               2021-22      3562.0923         565544.0   both  
      Clackmannanshire    2021-22       559.5432         115452.0   both  
      East Ayrshire       2010-11      3094.4776        1190983.0   both  
      Scotland            2019-20    148902.3909       52045545.0   both  
                          2021-22    170537.7903       26512151.0   both  
...                                          ...              ...    ...  
SW05  South Ayrshire      2021-22     30956.7301            887.5   both  
      South Lanarkshire   2021-22     76696.9263           1715.0   both  
      Stirling            2021-22     15465.3756            395.0   both  
      West Dunbartonshire 2021-22     21748.2469            480.0   both  
      West Lothian        2021-22     29981.5261            767.5   both  

[576 rows x 7 columns]

#We can do the same for the fully merged df (without separating new year's data)

This will give us all Data changes (any new data as well as any changes between old_df and new_df) 

In [92]:
diff_df = df.loc[(df['Indicator_old'] != df['Indicator_new'] )]
diff_df

Indicator_old  Numerator_old  \
Code   LocalAuthority   Year                                    
C&L01  Angus            2021-22         6.2791      3551.1012   
       Clackmannanshire 2021-22         5.2879       610.5016   
       East Ayrshire    2010-11         2.5983      3094.4776   
       Scotland         2019-20         2.8858    150191.5886   
                        2021-22         6.4339    170577.7577   
...                                        ...            ...   
CHN08b Scotland         2021-22            NaN            NaN   
CHN09  Scotland         2021-22            NaN            NaN   
CHN17  Scotland         2021-22            NaN            NaN   
CHN22  Scotland         2021-22            NaN            NaN   
CHN23  Scotland         2021-22            NaN            NaN   

                                 Denominator_old  Indicator_new  \
Code   LocalAuthority   Year                                      
C&L01  Angus            2021-22         565544.0         6.2985   
       Clackmannanshire 2021-22         115452.0         4.8465   
       East Ayrshire    2010-11        1190983.0         2.6003   
       Scotland         2019-20       52045545.0         2.8610   
                        2021-22       26512151.0         6.4324   
...                                          ...            ...   
CHN08b Scotland         2021-22              NaN       402.2039   
CHN09  Scotland         2021-22              NaN         0.8981   
CHN17  Scotland         2021-22              NaN         0.8214   
CHN22  Scotland         2021-22              NaN         0.0801   
CHN23  Scotland         2021-22              NaN         0.1586   

                                 Numerator_new  Denominator_new      _merge  
Code   LocalAuthority   Year                                                 
C&L01  Angus            2021-22      3562.0923         565544.0        both  
       Clackmannanshire 2021-22       559.5432         115452.0        both  
       East Ayrshire    2010-11      3094.4776        1190983.0        both  
       Scotland         2019-20    148902.3909       52045545.0        both  
                        2021-22    170537.7903       26512151.0        both  
...                                        ...              ...         ...  
CHN08b Scotland         2021-22    236586.8746          11312.0  right_only  
CHN09  Scotland         2021-22            NaN              NaN  right_only  
CHN17  Scotland         2021-22            NaN              NaN  right_only  
CHN22  Scotland         2021-22            NaN              NaN  right_only  
CHN23  Scotland         2021-22            NaN              NaN  right_only  

[774 rows x 7 columns]

#New year's data on top
Sort by 'merge' column so the newly added data is on top.

In [93]:
diff_df = diff_df.sort_values(by='_merge')
diff_df

Indicator_old  Numerator_old  \
Code   LocalAuthority      Year                                    
CHN23  Scotland            2021-22            NaN            NaN   
CHN09  Dundee City         2021-22            NaN            NaN   
       Dumfries & Galloway 2021-22            NaN            NaN   
       Clackmannanshire    2021-22            NaN            NaN   
       Argyll & Bute       2021-22            NaN            NaN   
...                                           ...            ...   
CHN08a North Lanarkshire   2010-11         3017.0            NaN   
                           2011-12         3288.0            NaN   
                           2012-13         3559.0            NaN   
       Perth & Kinross     2016-17         3664.0            NaN   
       West Lothian        2015-16         3462.0            NaN   

                                    Denominator_old  Indicator_new  \
Code   LocalAuthority      Year                                      
CHN23  Scotland            2021-22              NaN         0.1586   
CHN09  Dundee City         2021-22              NaN         0.8690   
       Dumfries & Galloway 2021-22              NaN         0.9274   
       Clackmannanshire    2021-22              NaN         0.9506   
       Argyll & Bute       2021-22              NaN         0.8194   
...                                             ...            ...   
CHN08a North Lanarkshire   2010-11              NaN      3017.3212   
                           2011-12              NaN      3288.2142   
                           2012-13              NaN      3558.6684   
       Perth & Kinross     2016-17              NaN      3663.5539   
       West Lothian        2015-16              NaN      3462.3366   

                                    Numerator_new  Denominator_new      _merge  
Code   LocalAuthority      Year                                                 
CHN23  Scotland            2021-22            NaN              NaN  right_only  
CHN09  Dundee City         2021-22            NaN              NaN  right_only  
       Dumfries & Galloway 2021-22            NaN              NaN  right_only  
       Clackmannanshire    2021-22            NaN              NaN  right_only  
       Argyll & Bute       2021-22            NaN              NaN  right_only  
...                                           ...              ...         ...  
CHN08a North Lanarkshire   2010-11            NaN              NaN        both  
                           2011-12            NaN              NaN        both  
                           2012-13            NaN              NaN        both  
       Perth & Kinross     2016-17            NaN              NaN        both  
       West Lothian        2015-16            NaN              NaN        both  

[774 rows x 7 columns]

#Rearrange columns so they have "old" and "new" columns next to each other.

In [94]:
diff_df = diff_df[['Indicator_old', 'Indicator_new' , 'Numerator_old',	'Numerator_new', 'Denominator_old',	'Denominator_new',	'_merge']]
diff_df

Indicator_old  Indicator_new  \
Code   LocalAuthority      Year                                    
CHN23  Scotland            2021-22            NaN         0.1586   
CHN09  Dundee City         2021-22            NaN         0.8690   
       Dumfries & Galloway 2021-22            NaN         0.9274   
       Clackmannanshire    2021-22            NaN         0.9506   
       Argyll & Bute       2021-22            NaN         0.8194   
...                                           ...            ...   
CHN08a North Lanarkshire   2010-11         3017.0      3017.3212   
                           2011-12         3288.0      3288.2142   
                           2012-13         3559.0      3558.6684   
       Perth & Kinross     2016-17         3664.0      3663.5539   
       West Lothian        2015-16         3462.0      3462.3366   

                                    Numerator_old  Numerator_new  \
Code   LocalAuthority      Year                                    
CHN23  Scotland            2021-22            NaN            NaN   
CHN09  Dundee City         2021-22            NaN            NaN   
       Dumfries & Galloway 2021-22            NaN            NaN   
       Clackmannanshire    2021-22            NaN            NaN   
       Argyll & Bute       2021-22            NaN            NaN   
...                                           ...            ...   
CHN08a North Lanarkshire   2010-11            NaN            NaN   
                           2011-12            NaN            NaN   
                           2012-13            NaN            NaN   
       Perth & Kinross     2016-17            NaN            NaN   
       West Lothian        2015-16            NaN            NaN   

                                    Denominator_old  Denominator_new  \
Code   LocalAuthority      Year                                        
CHN23  Scotland            2021-22              NaN              NaN   
CHN09  Dundee City         2021-22              NaN              NaN   
       Dumfries & Galloway 2021-22              NaN              NaN   
       Clackmannanshire    2021-22              NaN              NaN   
       Argyll & Bute       2021-22              NaN              NaN   
...                                             ...              ...   
CHN08a North Lanarkshire   2010-11              NaN              NaN   
                           2011-12              NaN              NaN   
                           2012-13              NaN              NaN   
       Perth & Kinross     2016-17              NaN              NaN   
       West Lothian        2015-16              NaN              NaN   

                                        _merge  
Code   LocalAuthority      Year                 
CHN23  Scotland            2021-22  right_only  
CHN09  Dundee City         2021-22  right_only  
       Dumfries & Galloway 2021-22  right_only  
       Clackmannanshire    2021-22  right_only  
       Argyll & Bute       2021-22  right_only  
...                                        ...  
CHN08a North Lanarkshire   2010-11        both  
                           2011-12        both  
                           2012-13        both  
       Perth & Kinross     2016-17        both  
       West Lothian        2015-16        both  

[774 rows x 7 columns]

#Download the file

In [ ]:
from google.colab import files
diff_df. to_excel('Data_changes.xlsx')
files.download('Data_changes.xlsx')

#Highlight changes in the new value columns

In [95]:
diff_df.columns[0]

'Indicator_old'

In [104]:
def set_colormap(x):
    c = f"background-color:green" 
    # DataFrame of styles
    df1 = pd.DataFrame('', index=x.index, columns=x.columns)
    #set new columns by condition
    columns = len(x.columns)
    for i in range (0,columns-1,2):
        col1 = x.columns[i]
        col2 = x.columns[i+1]
        m = x[col1]!=x[col2]
        # set columns by condition
        df1.loc[m, col2] = c
    return df1

highlight_df = diff_df.style.apply(set_colormap, axis=None)
     

Download the file

In [105]:
from google.colab import files
highlight_df. to_excel('Highlight_changes.xlsx')
files.download('Highlight_changes.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>